In [2]:
"""
Нужно подготовить код для обработки всех фото в датасете с сохранением разметки.
То есть, мы сохраняем новый файл фото с новым именем и пересохраняем разметку с таким же именем.
"""
from augmentation_instruments import pic_augmentation as aug
from augmentation_instruments import showImage, saveImage

import glob
import os
import random
import shutil
from functools import partial
from progress.bar import IncrementalBar

In [2]:
#открываем папку с относительным путём
path = "pics"
all_files = glob.glob(os.path.join(path,"*.jpg"))
#для каждого изображения в папке
#применяем каждый из фильтров и сохраняем изображения каждого из фильтров в отдельную папку
#вместе с фото переименовываем и переносим файл с разметкой в папку с дополненными данными

In [3]:
def get_img_data(img_path):
    file_root, file_name = os.path.split(img_path)
    img_name, img_ext = os.path.splitext(file_name)
    return file_root, img_name, img_ext

In [4]:
def isPath(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [9]:
def apply_augmentation(img_path, aug_type):
    """
    Применить аугментацию к изображению, если тип аугментации доступен
    и если есть файл *.txt с разметкой

    @img_path: относительный путь к изображению
    @aug_type: тип аугментации из числа "deblur", "gaus_blur", "sp_noise", "brightness", "contrast"
    """
    image = aug(img_path)

    img_root, image_name, _ = get_img_data(img_path)

    brightness_level = random.choice([-127, -64, 64, 127])
    contrast_level = random.choice([-64, 64])
    mask_size_level = random.choice([7,9,11,13])
    sp_noise_level = random.uniform(0.04,0.13)

    augmentation = {"deblur":partial(image.deblurring), 
                    "gaus_blur":partial(image.gaussian_blurring, mask_size=mask_size_level), 
                    "sp_noise":partial(image.sp_noise, sp_noise_level), 
                    "brightness":partial(image.apply_brightness_contrast, brightness=brightness_level), 
                    "contrast":partial(image.apply_brightness_contrast, contrast=contrast_level)}     
    
    augmented_image = augmentation[aug_type]()

    isPath(os.path.join(img_root,aug_type))
    # showImage(augmented_image)

    saveImage(augmented_image,
                os.path.join(img_root,aug_type),
                image_name,
                suffix="_"+aug_type)

    # пересохранить файл разметки с новым именем в папку к изображению
    shutil.copy(os.path.join(img_root,image_name)+".txt",
                os.path.join(img_root,aug_type,image_name)+"_"+aug_type+".txt")

# apply_augmentation(all_files[0], "deblurring")
# print("-------------")
# apply_augmentation(all_files[1], "gaussian_blurring")
# print("-------------")
# apply_augmentation(all_files[1], "sp_noise")
# print("-------------")
# apply_augmentation(all_files[1], "brightness")
# print("-------------")
# apply_augmentation(all_files[1], "contrast")



In [10]:
bar = IncrementalBar('Augmentation', max = len(all_files))

augmentation_types = {0:"deblur", 1:"gaus_blur", 2:"sp_noise", 3:"brightness", 4:"contrast"}

for img in all_files:
    # проверка на наличие разметки в данных
    img_root, image_name, _ = get_img_data(img)
    if os.path.exists(os.path.join(img_root,image_name)+".txt"): 
        # аугментация всех видов из списка
        for aug_type in augmentation_types.values():
            apply_augmentation(img, aug_type)
    else:
        print("No labels for the picture '"+image_name+".jpg'")

No labels for the picture '000006.jpg'
No labels for the picture '000007.jpg'
No labels for the picture '000008.jpg'
No labels for the picture '000009.jpg'
No labels for the picture '000010.jpg'
